In [2]:
%load_ext cudf.pandas
import pandas as pd
import json

/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:64: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


In [3]:
def read_json_file(file):
  with open(file) as f:
    data = json.load(f)
  return data

In [4]:
def make_dataframe_from_json(file):
  data = read_json_file(file)
  return pd.DataFrame(data)

In [5]:
df_with_query = make_dataframe_from_json('merged_final_malak.json')
df_with_query.head()

,prompt,meeting_transcript,summary
0,What action was taken regarding the motion to ...,Speaker 5: Not been withdrawn. We need to I th...,A motion to continue the item for one week was...
1,What decision was made regarding the entertain...,Speaker 4: Let's give them a round of applause...,The decision made regarding the entertainment ...
2,What was the outcome of the vote to approve am...,Speaker 4: See there's a motion in a second. M...,The vote to approve amendments to the terms an...
3,What are the key concerns raised by the public...,"Speaker 4: Well, thank you very much. And with...",Here's a summary of the key concerns raised an...
4,What were the key decisions made regarding the...,Speaker 0: Motion carries.\nSpeaker 4: Thank y...,The meeting transcript indicates that a motion...


In [6]:
def add_summerize_query(df):
  new_df = df.copy()
  new_df.loc[:, "prompt"] = "Summerize the Whole meeting"
  return new_df

In [7]:
splits = {'train': 'train.csv', 'validation': 'val.csv', 'test': 'test.csv'}
original_train_df = pd.read_csv("hf://datasets/lytang/MeetingBank-transcript/" + splits["train"])
original_train_df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


,meeting_id,source,type,reference,city
0,LongBeachCC_08092022_22-0922,Speaker 4: Thank you. And can we do the functi...,Agenda Item,Recommendation to increase appropriations in t...,LongBeachCC
1,LongBeachCC_08092022_22-0917,Speaker 4: We're going to hear all of the pres...,Public Hearing,Recommendation to conduct a Budget Hearing to ...,LongBeachCC
2,LongBeachCC_08092022_22-0946,Speaker 4: Thank you very much. We will. We're...,Ordinance,Recommendation to declare ordinance amending t...,LongBeachCC
3,LongBeachCC_08092022_22-0932,"Speaker 4: Great. Thank you. And, you know, we...",Resolution,Recommendation to adopt resolution approving t...,LongBeachCC
4,LongBeachCC_08092022_22-0926,Speaker 0: The motion is carried nine zero.\nS...,Agenda Item,Recommendation to refer to the Public Health a...,LongBeachCC


In [8]:
queried_original_df = add_summerize_query(original_train_df)
queried_original_df.head()

,meeting_id,source,type,reference,city,prompt
0,LongBeachCC_08092022_22-0922,Speaker 4: Thank you. And can we do the functi...,Agenda Item,Recommendation to increase appropriations in t...,LongBeachCC,Summerize the Whole meeting
1,LongBeachCC_08092022_22-0917,Speaker 4: We're going to hear all of the pres...,Public Hearing,Recommendation to conduct a Budget Hearing to ...,LongBeachCC,Summerize the Whole meeting
2,LongBeachCC_08092022_22-0946,Speaker 4: Thank you very much. We will. We're...,Ordinance,Recommendation to declare ordinance amending t...,LongBeachCC,Summerize the Whole meeting
3,LongBeachCC_08092022_22-0932,"Speaker 4: Great. Thank you. And, you know, we...",Resolution,Recommendation to adopt resolution approving t...,LongBeachCC,Summerize the Whole meeting
4,LongBeachCC_08092022_22-0926,Speaker 0: The motion is carried nine zero.\nS...,Agenda Item,Recommendation to refer to the Public Health a...,LongBeachCC,Summerize the Whole meeting


In [9]:
def renaming_dropping_columns(df):
  df = df.rename(columns={"source": "meeting_transcript", "reference": "summary"})
  df.drop(columns=["meeting_id", "type", "city"], inplace=True)
  return df

In [10]:
queried_original_df = renaming_dropping_columns(queried_original_df)
queried_original_df.head()

,meeting_transcript,summary,prompt
0,Speaker 4: Thank you. And can we do the functi...,Recommendation to increase appropriations in t...,Summerize the Whole meeting
1,Speaker 4: We're going to hear all of the pres...,Recommendation to conduct a Budget Hearing to ...,Summerize the Whole meeting
2,Speaker 4: Thank you very much. We will. We're...,Recommendation to declare ordinance amending t...,Summerize the Whole meeting
3,"Speaker 4: Great. Thank you. And, you know, we...",Recommendation to adopt resolution approving t...,Summerize the Whole meeting
4,Speaker 0: The motion is carried nine zero.\nS...,Recommendation to refer to the Public Health a...,Summerize the Whole meeting


In [11]:
from sklearn.model_selection import train_test_split

query_train, query_temp = train_test_split(df_with_query, test_size=0.3, random_state=42)
query_val, query_test = train_test_split(query_temp, test_size=0.5, random_state=42)

query_val = query_val.reset_index(drop=True)
query_test = query_test.reset_index(drop=True)


In [12]:
train_df = pd.concat([query_train, queried_original_df], axis=0)
train_df.head()

,prompt,meeting_transcript,summary
0,What are the key details of the proposal to tr...,"Speaker 0: Okay. Item 17, please.\nSpeaker 1: ...",The proposal involves transferring one-time di...
1,What specific actions and recommendations are ...,Speaker 0: The motion is here.\nSpeaker 1: Tha...,"Based on the meeting transcript, here's a summ..."
2,What specific monitoring and reporting measure...,Speaker 2: The Report of the Seattle Public Ut...,Here's a summary addressing the prompt:\n\n**M...
3,What was the outcome of the recommendation to ...,Speaker 0: Item number 34. Could you please be...,The recommendation to execute documents with D...
4,What conditions were placed on the entertainme...,Speaker 0: Thank you. We have a motion in a se...,The entertainment permit for St. Anita's resta...


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6149 entries, 0 to 6148
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   prompt              6149 non-null   object
 1   meeting_transcript  6149 non-null   object
 2   summary             6149 non-null   object
dtypes: object(3)
memory usage: 144.2+ KB


In [30]:
original_val_df = pd.read_csv("hf://datasets/lytang/MeetingBank-transcript/" + splits["validation"])

In [31]:
original_val_df = add_summerize_query(original_val_df)
final_val_df = renaming_dropping_columns(original_val_df)

In [32]:
final_val_df.info()
query_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   meeting_transcript  861 non-null    object
 1   summary             861 non-null    object
 2   prompt              861 non-null    object
dtypes: object(3)
memory usage: 20.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   prompt              210 non-null    object
 1   meeting_transcript  210 non-null    object
 2   summary             210 non-null    object
dtypes: object(3)
memory usage: 5.1+ KB


In [33]:
final_val_df = pd.concat([final_val_df, query_val], axis=0, ignore_index=True)

In [34]:
final_val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   meeting_transcript  1071 non-null   object
 1   summary             1071 non-null   object
 2   prompt              1071 non-null   object
dtypes: object(3)
memory usage: 25.2+ KB


In [38]:
original_test_df = pd.read_csv("hf://datasets/lytang/MeetingBank-transcript/" + splits["test"])

In [39]:
original_test_df = add_summerize_query(original_test_df)
final_test_df = renaming_dropping_columns(original_test_df)

In [40]:
final_test_df.info()
query_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   meeting_transcript  862 non-null    object
 1   summary             862 non-null    object
 2   prompt              862 non-null    object
dtypes: object(3)
memory usage: 20.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   prompt              210 non-null    object
 1   meeting_transcript  210 non-null    object
 2   summary             210 non-null    object
dtypes: object(3)
memory usage: 5.1+ KB


In [41]:
final_test_df = pd.concat([final_test_df, query_test], axis=0, ignore_index=True)

In [42]:
final_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   meeting_transcript  1072 non-null   object
 1   summary             1072 non-null   object
 2   prompt              1072 non-null   object
dtypes: object(3)
memory usage: 25.3+ KB


In [43]:
def save_df_in(df, path):
  df.to_parquet(f"{path}.parquet", index=False)

In [44]:
def save_df_as_chunks(df, path, chunk_size=1000):
  chunks = [df.iloc[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
  output_path = f"{path}_chunk"
  for i, chunk in enumerate(chunks):
    save_df_in(chunk, f"{output_path}_{i+1}")


In [45]:
save_df_as_chunks(train_df, "train")
save_df_in(final_val_df, "val")
save_df_in(final_test_df, "test")

In [46]:
chunk_size=1000
num_chunks = len(train_df) // chunk_size + 1
print("Number of Chunks: ", num_chunks)
for i in range(num_chunks):
  chunk = pd.read_parquet(f"train_chunk_{i+1}.parquet")
  print(f"Number of Rows in Chunk {i+1}: ", len(chunk))
print("Number of Rows in Original: ", len(train_df))

Number of Chunks:  7
Number of Rows in Chunk 1:  1000
Number of Rows in Chunk 2:  1000
Number of Rows in Chunk 3:  1000
Number of Rows in Chunk 4:  1000
Number of Rows in Chunk 5:  1000
Number of Rows in Chunk 6:  1000
Number of Rows in Chunk 7:  149
Number of Rows in Original:  6149
